In [1]:
import os
import sys
import pandas as pd
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

sys.path.append('/app/scripts/')

from HAM10000Dataset import HAM10000Dataset
from CrossValidationTrainer import CrossValidationTrainer

# Main directory
base_dir = '/app/'

# Data directory
data_dir = os.path.join(base_dir, "data")

# Directory for images
images_dir = os.path.join(data_dir, "HAM10000_images")

# Path to the metadata CSV file
metadata_file = os.path.join(data_dir, "HAM10000_metadata")

# Models directory
models_dir = os.path.join(base_dir, "models")

# Training data directory (for storing training plots and related data)
training_data_dir = os.path.join(base_dir, "training_data")

# Choose the model architecture
model_name = 'efficientnet_b0'

if model_name == 'resnet50':
    model_dir = os.path.join(models_dir, "resnet")
    training_data_output_dir = os.path.join(training_data_dir, "resnet")
elif model_name == 'densenet121':
    model_dir = os.path.join(models_dir, "densenet")
    training_data_output_dir = os.path.join(training_data_dir, "densenet")
elif model_name == 'efficientnet_b0':
    model_dir = os.path.join(models_dir, "efficientnet")
    training_data_output_dir = os.path.join(training_data_dir, "efficientnet")


# Step 1: Define transformations for validation (and potentially test) datasets
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to match the input size of the models
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Step 2: Define augmentations for the training dataset
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),          # 50% chance to apply horizontal flip
    transforms.RandomVerticalFlip(p=0.5),            # 50% chance to apply vertical flip
    transforms.RandomRotation(degrees=15),           # Rotate image by up to 15 degrees
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),  # Adjustments with more variation
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),  # Crop and resize with more aggressive scaling
    transforms.RandomAffine(
        degrees=0,                                   # No additional rotation
        translate=(0.1, 0.1),                        # Randomly translate by up to 10% of image dimensions
        scale=(0.9, 1.1),                            # Randomly scale image from 90% to 110%
        shear=10                                     # Shear by up to 10 degrees
    ),
    transforms.RandomPerspective(distortion_scale=0.1, p=0.5),  # Apply perspective transform with some distortion
    transforms.ToTensor(),                           # Convert image to tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])


# Step 3: Initialize the dataset with the image directory
dataset = HAM10000Dataset(
    annotations=pd.read_csv(metadata_file),          # Path to the metadata CSV file
    root_dir=images_dir,             # Directory containing images
    transform=val_transform,         # Use the validation transform by default
    augmentations=train_transform,   # Set the training augmentations
    balance_classes=True             # Use class balancing with WeightedRandomSampler
)

# Step 4: Visualize class distribution and example images (Optional)
# dataset.show_class_distribution()
# dataset.show_examples(num_images=5)

# Step 5: Initialize the CrossValidationTrainer with the dataset
trainer = CrossValidationTrainer(
    dataset=dataset,
    model_name=model_name,          # Choose model architecture: "resnet50", "densenet121", or "efficientnet_b0"
    num_classes=7,                  # HAM10000 has 7 classes
    num_folds=5,                    # Number of folds for cross-validation
    batch_size=32,                  # Batch size for DataLoader
    num_epochs=1000,                  # Number of epochs for each fold
    lr=0.001,                       # Learning rate
    patience=10,                    # Patience for early stopping
    train_transform=train_transform, # Apply augmentations during training
    val_transform=val_transform      # Apply only resizing and normalization during validation
)

# Step 6: Perform cross-validation
results_df, best_models = trainer.cross_validate()

# Step 7: Save cross-validation results
results_df.to_csv(os.path.join(training_data_output_dir, "cross_validation_results_ham10000.csv"), index=False)

# Save the best model from each fold
for i, model_state in enumerate(best_models):
    torch.save(model_state, os.path.join(model_dir, f"best_model_ham10000_fold_{i+1}.pth"))


Using device: cuda
FOLD 1/5


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 36.0MB/s]



Epoch 1/1000


Training: 100%|██████████| 251/251 [01:20<00:00,  3.12it/s]


Epoch 1 Training: Loss: 0.8320, Accuracy: 0.6995


...

Epoch 24 Validation: Loss: 0.4013, Accuracy: 0.8672
Learning Rate: 1.0000000000000002e-06

Epoch 25/1000


Training: 100%|██████████| 251/251 [03:11<00:00,  1.31it/s]


Epoch 25 Training: Loss: 0.0981, Accuracy: 0.9657


Validating: 100%|██████████| 63/63 [00:33<00:00,  1.91it/s]


Epoch 25 Validation: Loss: 0.3931, Accuracy: 0.8717
Learning Rate: 1.0000000000000002e-06
Early stopping at epoch 25 with best validation loss: 0.3810
